# Real Hardware Fitting

In [1]:
import sys
sys.path.insert(0, '../../src/')

import numpy as np
import qiskit as qk
import matplotlib.pyplot as plt
import multiprocessing as mp
import random

import pickle
from qiskit.quantum_info import DensityMatrix
from qiskit.quantum_info import Operator
from scipy.linalg import sqrtm
from tqdm.notebook import tqdm

from src import *
#np.set_printoptions(threshold=sys.maxsize)

## Real Hardware

### Two Qubits, Santiago

#### One Block

In [3]:
input_list, target_list = pickle.load(open("../../data/hsc_single.p", "rb"))

n = 2
d = 2**n
rank = 3

circuit = qk.QuantumCircuit(2)
circuit.h(0)
circuit.s(1)
circuit.cnot(0,1)

U = Operator(circuit).data

kraus_model = KrausMap(U, 0.5, d, rank)

model1 = ModelQuantumMap(q_map = kraus_model,
                         cost = state_density_loss,
                         input_list = input_list,
                         target_list = target_list,
                         lr = 0.05, 
                         h = 1e-3)

model1.train(num_iter = 2000, 
             use_adam = True)

  0%|          | 0/2000 [00:00<?, ?it/s]

0: fid: -0.759, c: 0.512
1: fid: -0.771, c: 0.525
2: fid: -0.781, c: 0.537
3: fid: -0.792, c: 0.549
4: fid: -0.802, c: 0.560
5: fid: -0.811, c: 0.572
6: fid: -0.820, c: 0.584
7: fid: -0.828, c: 0.596
8: fid: -0.836, c: 0.608
9: fid: -0.844, c: 0.619
10: fid: -0.852, c: 0.631
11: fid: -0.859, c: 0.642
12: fid: -0.866, c: 0.653
13: fid: -0.873, c: 0.664
14: fid: -0.879, c: 0.675
15: fid: -0.886, c: 0.685
16: fid: -0.891, c: 0.695
17: fid: -0.897, c: 0.705
18: fid: -0.902, c: 0.714
19: fid: -0.906, c: 0.722
20: fid: -0.911, c: 0.731
21: fid: -0.915, c: 0.739
22: fid: -0.919, c: 0.747
23: fid: -0.922, c: 0.754
24: fid: -0.926, c: 0.762
25: fid: -0.929, c: 0.769
26: fid: -0.932, c: 0.777
27: fid: -0.935, c: 0.783
28: fid: -0.938, c: 0.790
29: fid: -0.940, c: 0.796
30: fid: -0.942, c: 0.801
31: fid: -0.944, c: 0.807
32: fid: -0.946, c: 0.812
33: fid: -0.948, c: 0.817
34: fid: -0.950, c: 0.823
35: fid: -0.951, c: 0.827
36: fid: -0.953, c: 0.832
37: fid: -0.954, c: 0.836
38: fid: -0.955, c: 0.

309: fid: -0.981, c: 0.909
310: fid: -0.981, c: 0.908
311: fid: -0.981, c: 0.908
312: fid: -0.981, c: 0.907
313: fid: -0.981, c: 0.907
314: fid: -0.981, c: 0.907
315: fid: -0.981, c: 0.906
316: fid: -0.981, c: 0.906
317: fid: -0.981, c: 0.906
318: fid: -0.981, c: 0.905
319: fid: -0.981, c: 0.905
320: fid: -0.981, c: 0.905
321: fid: -0.981, c: 0.904
322: fid: -0.982, c: 0.904
323: fid: -0.982, c: 0.904
324: fid: -0.982, c: 0.903
325: fid: -0.982, c: 0.903
326: fid: -0.982, c: 0.902
327: fid: -0.982, c: 0.902
328: fid: -0.982, c: 0.902
329: fid: -0.982, c: 0.902
330: fid: -0.982, c: 0.902
331: fid: -0.982, c: 0.902
332: fid: -0.982, c: 0.902
333: fid: -0.982, c: 0.902
334: fid: -0.982, c: 0.902
335: fid: -0.982, c: 0.902
336: fid: -0.982, c: 0.902
337: fid: -0.982, c: 0.901
338: fid: -0.982, c: 0.901
339: fid: -0.982, c: 0.901
340: fid: -0.982, c: 0.901
341: fid: -0.982, c: 0.901
342: fid: -0.982, c: 0.901
343: fid: -0.982, c: 0.900
344: fid: -0.982, c: 0.900
345: fid: -0.982, c: 0.900
3

614: fid: -0.985, c: 0.860
615: fid: -0.985, c: 0.859
616: fid: -0.985, c: 0.858
617: fid: -0.985, c: 0.858
618: fid: -0.985, c: 0.857
619: fid: -0.985, c: 0.857
620: fid: -0.985, c: 0.857
621: fid: -0.985, c: 0.857
622: fid: -0.985, c: 0.857
623: fid: -0.985, c: 0.857
624: fid: -0.985, c: 0.857
625: fid: -0.985, c: 0.857
626: fid: -0.985, c: 0.858
627: fid: -0.985, c: 0.858
628: fid: -0.985, c: 0.859
629: fid: -0.985, c: 0.859
630: fid: -0.985, c: 0.860
631: fid: -0.985, c: 0.860
632: fid: -0.985, c: 0.861
633: fid: -0.985, c: 0.861
634: fid: -0.985, c: 0.861
635: fid: -0.985, c: 0.862
636: fid: -0.985, c: 0.862
637: fid: -0.985, c: 0.862
638: fid: -0.985, c: 0.862
639: fid: -0.985, c: 0.862
640: fid: -0.985, c: 0.862
641: fid: -0.985, c: 0.862
642: fid: -0.985, c: 0.861
643: fid: -0.985, c: 0.861
644: fid: -0.985, c: 0.861
645: fid: -0.985, c: 0.861
646: fid: -0.985, c: 0.861
647: fid: -0.985, c: 0.861
648: fid: -0.985, c: 0.861
649: fid: -0.985, c: 0.861
650: fid: -0.985, c: 0.860
6

919: fid: -0.987, c: 0.757
920: fid: -0.987, c: 0.756
921: fid: -0.987, c: 0.755
922: fid: -0.987, c: 0.755
923: fid: -0.987, c: 0.754
924: fid: -0.987, c: 0.753
925: fid: -0.987, c: 0.752
926: fid: -0.987, c: 0.750
927: fid: -0.987, c: 0.749
928: fid: -0.987, c: 0.748
929: fid: -0.987, c: 0.747
930: fid: -0.987, c: 0.746
931: fid: -0.986, c: 0.745
932: fid: -0.986, c: 0.744
933: fid: -0.986, c: 0.743
934: fid: -0.986, c: 0.742
935: fid: -0.986, c: 0.741
936: fid: -0.986, c: 0.741
937: fid: -0.986, c: 0.740
938: fid: -0.986, c: 0.740
939: fid: -0.986, c: 0.738
940: fid: -0.986, c: 0.737
941: fid: -0.986, c: 0.736
942: fid: -0.986, c: 0.736
943: fid: -0.986, c: 0.736
944: fid: -0.986, c: 0.736
945: fid: -0.986, c: 0.736
946: fid: -0.986, c: 0.736
947: fid: -0.987, c: 0.737
948: fid: -0.987, c: 0.738
949: fid: -0.987, c: 0.737
950: fid: -0.987, c: 0.737
951: fid: -0.987, c: 0.737
952: fid: -0.987, c: 0.737
953: fid: -0.987, c: 0.736
954: fid: -0.987, c: 0.735
955: fid: -0.987, c: 0.735
9

1217: fid: -0.987, c: 0.667
1218: fid: -0.988, c: 0.667
1219: fid: -0.987, c: 0.667
1220: fid: -0.988, c: 0.666
1221: fid: -0.988, c: 0.665
1222: fid: -0.988, c: 0.664
1223: fid: -0.988, c: 0.663
1224: fid: -0.988, c: 0.662
1225: fid: -0.988, c: 0.661
1226: fid: -0.988, c: 0.659
1227: fid: -0.988, c: 0.658
1228: fid: -0.988, c: 0.657
1229: fid: -0.988, c: 0.656
1230: fid: -0.988, c: 0.656
1231: fid: -0.988, c: 0.654
1232: fid: -0.988, c: 0.654
1233: fid: -0.988, c: 0.653
1234: fid: -0.988, c: 0.651
1235: fid: -0.988, c: 0.650
1236: fid: -0.988, c: 0.649
1237: fid: -0.988, c: 0.649
1238: fid: -0.988, c: 0.648
1239: fid: -0.988, c: 0.647
1240: fid: -0.988, c: 0.646
1241: fid: -0.989, c: 0.645
1242: fid: -0.989, c: 0.644
1243: fid: -0.989, c: 0.644
1244: fid: -0.989, c: 0.643
1245: fid: -0.989, c: 0.642
1246: fid: -0.989, c: 0.641
1247: fid: -0.989, c: 0.640
1248: fid: -0.989, c: 0.639
1249: fid: -0.989, c: 0.638
1250: fid: -0.989, c: 0.638
1251: fid: -0.989, c: 0.637
1252: fid: -0.989, c

1511: fid: -0.989, c: 0.589
1512: fid: -0.989, c: 0.589
1513: fid: -0.989, c: 0.588
1514: fid: -0.989, c: 0.587
1515: fid: -0.989, c: 0.587
1516: fid: -0.989, c: 0.587
1517: fid: -0.989, c: 0.587
1518: fid: -0.989, c: 0.588
1519: fid: -0.989, c: 0.589
1520: fid: -0.989, c: 0.589
1521: fid: -0.989, c: 0.590
1522: fid: -0.989, c: 0.591
1523: fid: -0.989, c: 0.592
1524: fid: -0.989, c: 0.593
1525: fid: -0.989, c: 0.594
1526: fid: -0.989, c: 0.595
1527: fid: -0.989, c: 0.596
1528: fid: -0.989, c: 0.597
1529: fid: -0.989, c: 0.598
1530: fid: -0.989, c: 0.599
1531: fid: -0.989, c: 0.599
1532: fid: -0.989, c: 0.600
1533: fid: -0.989, c: 0.601
1534: fid: -0.989, c: 0.603
1535: fid: -0.989, c: 0.603
1536: fid: -0.989, c: 0.603
1537: fid: -0.989, c: 0.604
1538: fid: -0.989, c: 0.603
1539: fid: -0.989, c: 0.602
1540: fid: -0.989, c: 0.601
1541: fid: -0.989, c: 0.600
1542: fid: -0.989, c: 0.600
1543: fid: -0.989, c: 0.600
1544: fid: -0.989, c: 0.600
1545: fid: -0.989, c: 0.599
1546: fid: -0.989, c

1805: fid: -0.989, c: 0.544
1806: fid: -0.989, c: 0.543
1807: fid: -0.989, c: 0.541
1808: fid: -0.989, c: 0.539
1809: fid: -0.989, c: 0.537
1810: fid: -0.989, c: 0.536
1811: fid: -0.989, c: 0.534
1812: fid: -0.989, c: 0.533
1813: fid: -0.989, c: 0.533
1814: fid: -0.989, c: 0.533
1815: fid: -0.989, c: 0.533
1816: fid: -0.989, c: 0.533
1817: fid: -0.989, c: 0.533
1818: fid: -0.989, c: 0.534
1819: fid: -0.989, c: 0.534
1820: fid: -0.989, c: 0.535
1821: fid: -0.989, c: 0.536
1822: fid: -0.989, c: 0.537
1823: fid: -0.989, c: 0.538
1824: fid: -0.989, c: 0.539
1825: fid: -0.989, c: 0.540
1826: fid: -0.989, c: 0.540
1827: fid: -0.989, c: 0.540
1828: fid: -0.989, c: 0.541
1829: fid: -0.990, c: 0.541
1830: fid: -0.990, c: 0.542
1831: fid: -0.990, c: 0.543
1832: fid: -0.990, c: 0.543
1833: fid: -0.990, c: 0.543
1834: fid: -0.990, c: 0.544
1835: fid: -0.990, c: 0.544
1836: fid: -0.990, c: 0.544
1837: fid: -0.990, c: 0.544
1838: fid: -0.990, c: 0.544
1839: fid: -0.990, c: 0.543
1840: fid: -0.990, c

#### Two Blocks

In [4]:
input_list, target_list = pickle.load(open("../../data/hsc_double.p", "rb"))

n = 2
d = 2**n
rank = 4

circuit = qk.QuantumCircuit(2)
circuit.h(0)
circuit.s(1)
circuit.cnot(0,1)
circuit.h(0)
circuit.s(1)
circuit.cnot(0,1)

U = Operator(circuit).data

kraus_model = KrausMap(U, 0.5, d, rank)

model2 = ModelQuantumMap(q_map = kraus_model,
                         cost = state_density_loss,
                         input_list = input_list,
                         target_list = target_list,
                         lr = 0.05, 
                         h = 1e-3)

model2.train(num_iter = 2000, 
             use_adam = True)

  0%|          | 0/2000 [00:00<?, ?it/s]

0: fid: -0.804, c: 0.512
1: fid: -0.813, c: 0.525
2: fid: -0.822, c: 0.537
3: fid: -0.830, c: 0.550
4: fid: -0.838, c: 0.562
5: fid: -0.846, c: 0.574
6: fid: -0.854, c: 0.586
7: fid: -0.861, c: 0.598
8: fid: -0.868, c: 0.610
9: fid: -0.875, c: 0.622
10: fid: -0.882, c: 0.633
11: fid: -0.888, c: 0.644
12: fid: -0.894, c: 0.655
13: fid: -0.899, c: 0.665
14: fid: -0.904, c: 0.675
15: fid: -0.909, c: 0.684
16: fid: -0.913, c: 0.692
17: fid: -0.917, c: 0.701
18: fid: -0.921, c: 0.710
19: fid: -0.925, c: 0.718
20: fid: -0.928, c: 0.726
21: fid: -0.931, c: 0.733
22: fid: -0.934, c: 0.741
23: fid: -0.937, c: 0.748
24: fid: -0.939, c: 0.754
25: fid: -0.941, c: 0.760
26: fid: -0.943, c: 0.765
27: fid: -0.945, c: 0.770
28: fid: -0.946, c: 0.775
29: fid: -0.948, c: 0.779
30: fid: -0.949, c: 0.784
31: fid: -0.950, c: 0.789
32: fid: -0.952, c: 0.793
33: fid: -0.953, c: 0.797
34: fid: -0.954, c: 0.801
35: fid: -0.955, c: 0.805
36: fid: -0.956, c: 0.808
37: fid: -0.956, c: 0.812
38: fid: -0.957, c: 0.

309: fid: -0.982, c: 0.762
310: fid: -0.982, c: 0.761
311: fid: -0.982, c: 0.760
312: fid: -0.982, c: 0.760
313: fid: -0.982, c: 0.759
314: fid: -0.982, c: 0.759
315: fid: -0.982, c: 0.759
316: fid: -0.982, c: 0.759
317: fid: -0.982, c: 0.758
318: fid: -0.982, c: 0.758
319: fid: -0.982, c: 0.757
320: fid: -0.982, c: 0.757
321: fid: -0.982, c: 0.756
322: fid: -0.982, c: 0.756
323: fid: -0.982, c: 0.755
324: fid: -0.982, c: 0.755
325: fid: -0.982, c: 0.754
326: fid: -0.982, c: 0.753
327: fid: -0.982, c: 0.753
328: fid: -0.982, c: 0.752
329: fid: -0.982, c: 0.751
330: fid: -0.982, c: 0.751
331: fid: -0.982, c: 0.751
332: fid: -0.982, c: 0.750
333: fid: -0.982, c: 0.750
334: fid: -0.983, c: 0.749
335: fid: -0.983, c: 0.747
336: fid: -0.983, c: 0.746
337: fid: -0.983, c: 0.745
338: fid: -0.983, c: 0.744
339: fid: -0.983, c: 0.743
340: fid: -0.983, c: 0.741
341: fid: -0.983, c: 0.740
342: fid: -0.983, c: 0.738
343: fid: -0.983, c: 0.737
344: fid: -0.983, c: 0.736
345: fid: -0.983, c: 0.735
3

614: fid: -0.989, c: 0.431
615: fid: -0.989, c: 0.430
616: fid: -0.989, c: 0.430
617: fid: -0.989, c: 0.430
618: fid: -0.989, c: 0.429
619: fid: -0.989, c: 0.429
620: fid: -0.989, c: 0.428
621: fid: -0.989, c: 0.427
622: fid: -0.990, c: 0.427
623: fid: -0.990, c: 0.427
624: fid: -0.990, c: 0.427
625: fid: -0.990, c: 0.426
626: fid: -0.990, c: 0.426
627: fid: -0.990, c: 0.426
628: fid: -0.990, c: 0.426
629: fid: -0.990, c: 0.425
630: fid: -0.990, c: 0.425
631: fid: -0.990, c: 0.425
632: fid: -0.990, c: 0.424
633: fid: -0.990, c: 0.424
634: fid: -0.990, c: 0.424
635: fid: -0.990, c: 0.423
636: fid: -0.990, c: 0.422
637: fid: -0.990, c: 0.422
638: fid: -0.990, c: 0.421
639: fid: -0.990, c: 0.420
640: fid: -0.990, c: 0.419
641: fid: -0.990, c: 0.418
642: fid: -0.990, c: 0.417
643: fid: -0.990, c: 0.416
644: fid: -0.990, c: 0.414
645: fid: -0.990, c: 0.413
646: fid: -0.990, c: 0.412
647: fid: -0.990, c: 0.410
648: fid: -0.990, c: 0.409
649: fid: -0.989, c: 0.407
650: fid: -0.989, c: 0.406
6

920: fid: -0.990, c: 0.343
921: fid: -0.990, c: 0.342
922: fid: -0.990, c: 0.342
923: fid: -0.990, c: 0.342
924: fid: -0.990, c: 0.341
925: fid: -0.990, c: 0.340
926: fid: -0.990, c: 0.339
927: fid: -0.990, c: 0.339
928: fid: -0.990, c: 0.338
929: fid: -0.990, c: 0.337
930: fid: -0.990, c: 0.336
931: fid: -0.990, c: 0.335
932: fid: -0.990, c: 0.334
933: fid: -0.990, c: 0.333
934: fid: -0.990, c: 0.332
935: fid: -0.990, c: 0.331
936: fid: -0.990, c: 0.330
937: fid: -0.990, c: 0.330
938: fid: -0.990, c: 0.329
939: fid: -0.991, c: 0.328
940: fid: -0.991, c: 0.327
941: fid: -0.991, c: 0.326
942: fid: -0.991, c: 0.326
943: fid: -0.991, c: 0.325
944: fid: -0.991, c: 0.324
945: fid: -0.990, c: 0.323
946: fid: -0.990, c: 0.322
947: fid: -0.990, c: 0.321
948: fid: -0.990, c: 0.321
949: fid: -0.990, c: 0.320
950: fid: -0.990, c: 0.320
951: fid: -0.990, c: 0.320
952: fid: -0.990, c: 0.320
953: fid: -0.990, c: 0.319
954: fid: -0.990, c: 0.319
955: fid: -0.990, c: 0.319
956: fid: -0.990, c: 0.318
9

1216: fid: -0.991, c: 0.290
1217: fid: -0.991, c: 0.290
1218: fid: -0.990, c: 0.291
1219: fid: -0.990, c: 0.291
1220: fid: -0.990, c: 0.291
1221: fid: -0.990, c: 0.291
1222: fid: -0.991, c: 0.291
1223: fid: -0.991, c: 0.291
1224: fid: -0.991, c: 0.292
1225: fid: -0.991, c: 0.293
1226: fid: -0.991, c: 0.294
1227: fid: -0.991, c: 0.294
1228: fid: -0.991, c: 0.295
1229: fid: -0.991, c: 0.296
1230: fid: -0.991, c: 0.296
1231: fid: -0.991, c: 0.297
1232: fid: -0.991, c: 0.297
1233: fid: -0.991, c: 0.298
1234: fid: -0.991, c: 0.298
1235: fid: -0.991, c: 0.299
1236: fid: -0.991, c: 0.299
1237: fid: -0.991, c: 0.300
1238: fid: -0.991, c: 0.301
1239: fid: -0.991, c: 0.301
1240: fid: -0.991, c: 0.301
1241: fid: -0.991, c: 0.301
1242: fid: -0.991, c: 0.301
1243: fid: -0.991, c: 0.301
1244: fid: -0.991, c: 0.301
1245: fid: -0.991, c: 0.301
1246: fid: -0.991, c: 0.301
1247: fid: -0.991, c: 0.301
1248: fid: -0.991, c: 0.301
1249: fid: -0.991, c: 0.300
1250: fid: -0.991, c: 0.300
1251: fid: -0.991, c

1511: fid: -0.991, c: 0.295
1512: fid: -0.991, c: 0.295
1513: fid: -0.991, c: 0.295
1514: fid: -0.991, c: 0.295
1515: fid: -0.991, c: 0.295
1516: fid: -0.991, c: 0.295
1517: fid: -0.991, c: 0.295
1518: fid: -0.991, c: 0.295
1519: fid: -0.990, c: 0.295
1520: fid: -0.990, c: 0.295
1521: fid: -0.990, c: 0.294
1522: fid: -0.990, c: 0.294
1523: fid: -0.990, c: 0.294
1524: fid: -0.990, c: 0.293
1525: fid: -0.990, c: 0.293
1526: fid: -0.990, c: 0.293
1527: fid: -0.990, c: 0.292
1528: fid: -0.991, c: 0.292
1529: fid: -0.991, c: 0.291
1530: fid: -0.991, c: 0.291
1531: fid: -0.991, c: 0.291
1532: fid: -0.991, c: 0.291
1533: fid: -0.991, c: 0.291
1534: fid: -0.991, c: 0.290
1535: fid: -0.991, c: 0.289
1536: fid: -0.991, c: 0.288
1537: fid: -0.991, c: 0.287
1538: fid: -0.991, c: 0.286
1539: fid: -0.991, c: 0.284
1540: fid: -0.991, c: 0.284
1541: fid: -0.991, c: 0.283
1542: fid: -0.991, c: 0.283
1543: fid: -0.991, c: 0.283
1544: fid: -0.991, c: 0.282
1545: fid: -0.991, c: 0.281
1546: fid: -0.991, c

1806: fid: -0.991, c: 0.286
1807: fid: -0.991, c: 0.286
1808: fid: -0.991, c: 0.286
1809: fid: -0.991, c: 0.286
1810: fid: -0.991, c: 0.286
1811: fid: -0.991, c: 0.286
1812: fid: -0.991, c: 0.286
1813: fid: -0.991, c: 0.285
1814: fid: -0.991, c: 0.285
1815: fid: -0.991, c: 0.285
1816: fid: -0.991, c: 0.285
1817: fid: -0.991, c: 0.285
1818: fid: -0.991, c: 0.284
1819: fid: -0.991, c: 0.285
1820: fid: -0.991, c: 0.284
1821: fid: -0.991, c: 0.283
1822: fid: -0.991, c: 0.283
1823: fid: -0.991, c: 0.282
1824: fid: -0.991, c: 0.282
1825: fid: -0.991, c: 0.281
1826: fid: -0.991, c: 0.281
1827: fid: -0.991, c: 0.282
1828: fid: -0.991, c: 0.282
1829: fid: -0.991, c: 0.282
1830: fid: -0.991, c: 0.283
1831: fid: -0.991, c: 0.284
1832: fid: -0.991, c: 0.284
1833: fid: -0.991, c: 0.285
1834: fid: -0.991, c: 0.285
1835: fid: -0.991, c: 0.285
1836: fid: -0.991, c: 0.285
1837: fid: -0.991, c: 0.284
1838: fid: -0.992, c: 0.284
1839: fid: -0.992, c: 0.284
1840: fid: -0.992, c: 0.283
1841: fid: -0.992, c

#### Three Blocks

In [5]:
input_list, target_list = pickle.load(open("../../data/hsc_triple.p", "rb"))

n = 2
d = 2**n
rank = 5

circuit = qk.QuantumCircuit(2)
circuit.h(0)
circuit.s(1)
circuit.cnot(0,1)
circuit.h(0)
circuit.s(1)
circuit.cnot(0,1)
circuit.h(0)
circuit.s(1)
circuit.cnot(0,1)

U = Operator(circuit).data

kraus_model = KrausMap(U, 0.5, d, rank)

model3 = ModelQuantumMap(q_map = kraus_model,
                         cost = state_density_loss,
                         input_list = input_list,
                         target_list = target_list,
                         lr = 0.05, 
                         h = 1e-3)

model3.train(num_iter = 2000, 
             use_adam = True)

  0%|          | 0/2000 [00:00<?, ?it/s]

0: fid: -0.798, c: 0.512
1: fid: -0.808, c: 0.525
2: fid: -0.817, c: 0.537
3: fid: -0.826, c: 0.550
4: fid: -0.834, c: 0.562
5: fid: -0.842, c: 0.574
6: fid: -0.850, c: 0.586
7: fid: -0.857, c: 0.598
8: fid: -0.864, c: 0.609
9: fid: -0.871, c: 0.621
10: fid: -0.878, c: 0.632
11: fid: -0.884, c: 0.644
12: fid: -0.890, c: 0.655
13: fid: -0.896, c: 0.665
14: fid: -0.901, c: 0.676
15: fid: -0.906, c: 0.686
16: fid: -0.910, c: 0.695
17: fid: -0.915, c: 0.704
18: fid: -0.919, c: 0.713
19: fid: -0.922, c: 0.721
20: fid: -0.926, c: 0.729
21: fid: -0.929, c: 0.736
22: fid: -0.932, c: 0.744
23: fid: -0.935, c: 0.751
24: fid: -0.938, c: 0.758
25: fid: -0.940, c: 0.765
26: fid: -0.942, c: 0.771
27: fid: -0.944, c: 0.777
28: fid: -0.946, c: 0.782
29: fid: -0.948, c: 0.787
30: fid: -0.949, c: 0.792
31: fid: -0.950, c: 0.796
32: fid: -0.951, c: 0.800
33: fid: -0.952, c: 0.804
34: fid: -0.953, c: 0.808
35: fid: -0.954, c: 0.812
36: fid: -0.955, c: 0.815
37: fid: -0.956, c: 0.818
38: fid: -0.957, c: 0.

308: fid: -0.984, c: 0.639
309: fid: -0.984, c: 0.637
310: fid: -0.984, c: 0.634
311: fid: -0.984, c: 0.632
312: fid: -0.985, c: 0.629
313: fid: -0.985, c: 0.627
314: fid: -0.985, c: 0.624
315: fid: -0.985, c: 0.622
316: fid: -0.985, c: 0.619
317: fid: -0.985, c: 0.617
318: fid: -0.985, c: 0.614
319: fid: -0.985, c: 0.612
320: fid: -0.985, c: 0.609
321: fid: -0.985, c: 0.607
322: fid: -0.985, c: 0.604
323: fid: -0.985, c: 0.602
324: fid: -0.985, c: 0.599
325: fid: -0.985, c: 0.597
326: fid: -0.985, c: 0.595
327: fid: -0.986, c: 0.592
328: fid: -0.986, c: 0.590
329: fid: -0.986, c: 0.587
330: fid: -0.986, c: 0.584
331: fid: -0.986, c: 0.581
332: fid: -0.986, c: 0.578
333: fid: -0.986, c: 0.576
334: fid: -0.986, c: 0.573
335: fid: -0.986, c: 0.570
336: fid: -0.986, c: 0.568
337: fid: -0.986, c: 0.565
338: fid: -0.986, c: 0.562
339: fid: -0.986, c: 0.559
340: fid: -0.986, c: 0.556
341: fid: -0.987, c: 0.553
342: fid: -0.987, c: 0.550
343: fid: -0.987, c: 0.547
344: fid: -0.987, c: 0.544
3

612: fid: -0.992, c: 0.231
613: fid: -0.993, c: 0.230
614: fid: -0.993, c: 0.230
615: fid: -0.993, c: 0.230
616: fid: -0.993, c: 0.229
617: fid: -0.993, c: 0.229
618: fid: -0.993, c: 0.229
619: fid: -0.993, c: 0.228
620: fid: -0.993, c: 0.228
621: fid: -0.993, c: 0.227
622: fid: -0.993, c: 0.226
623: fid: -0.993, c: 0.225
624: fid: -0.992, c: 0.225
625: fid: -0.992, c: 0.224
626: fid: -0.992, c: 0.223
627: fid: -0.992, c: 0.223
628: fid: -0.991, c: 0.222
629: fid: -0.991, c: 0.222
630: fid: -0.991, c: 0.221
631: fid: -0.991, c: 0.221
632: fid: -0.991, c: 0.221
633: fid: -0.991, c: 0.221
634: fid: -0.992, c: 0.221
635: fid: -0.992, c: 0.221
636: fid: -0.992, c: 0.221
637: fid: -0.992, c: 0.221
638: fid: -0.992, c: 0.221
639: fid: -0.992, c: 0.221
640: fid: -0.992, c: 0.221
641: fid: -0.993, c: 0.221
642: fid: -0.993, c: 0.221
643: fid: -0.993, c: 0.220
644: fid: -0.993, c: 0.220
645: fid: -0.993, c: 0.220
646: fid: -0.993, c: 0.219
647: fid: -0.993, c: 0.219
648: fid: -0.993, c: 0.219
6

916: fid: -0.994, c: 0.167
917: fid: -0.994, c: 0.167
918: fid: -0.994, c: 0.167
919: fid: -0.995, c: 0.166
920: fid: -0.995, c: 0.166
921: fid: -0.995, c: 0.166
922: fid: -0.995, c: 0.165
923: fid: -0.994, c: 0.165
924: fid: -0.994, c: 0.165
925: fid: -0.994, c: 0.165
926: fid: -0.993, c: 0.165
927: fid: -0.993, c: 0.165
928: fid: -0.993, c: 0.165
929: fid: -0.993, c: 0.165
930: fid: -0.993, c: 0.165
931: fid: -0.993, c: 0.165
932: fid: -0.993, c: 0.166
933: fid: -0.993, c: 0.166
934: fid: -0.993, c: 0.166
935: fid: -0.993, c: 0.167
936: fid: -0.993, c: 0.167
937: fid: -0.993, c: 0.167
938: fid: -0.993, c: 0.167
939: fid: -0.993, c: 0.167
940: fid: -0.993, c: 0.167
941: fid: -0.993, c: 0.167
942: fid: -0.993, c: 0.167
943: fid: -0.993, c: 0.167
944: fid: -0.993, c: 0.167
945: fid: -0.993, c: 0.167
946: fid: -0.993, c: 0.167
947: fid: -0.993, c: 0.167
948: fid: -0.993, c: 0.167
949: fid: -0.993, c: 0.167
950: fid: -0.993, c: 0.167
951: fid: -0.993, c: 0.167
952: fid: -0.993, c: 0.167
9

1212: fid: -0.994, c: 0.151
1213: fid: -0.994, c: 0.150
1214: fid: -0.994, c: 0.150
1215: fid: -0.994, c: 0.150
1216: fid: -0.994, c: 0.150
1217: fid: -0.994, c: 0.150
1218: fid: -0.994, c: 0.150
1219: fid: -0.994, c: 0.150
1220: fid: -0.995, c: 0.150
1221: fid: -0.995, c: 0.151
1222: fid: -0.994, c: 0.151
1223: fid: -0.994, c: 0.151
1224: fid: -0.994, c: 0.151
1225: fid: -0.994, c: 0.151
1226: fid: -0.994, c: 0.152
1227: fid: -0.994, c: 0.152
1228: fid: -0.994, c: 0.153
1229: fid: -0.994, c: 0.153
1230: fid: -0.994, c: 0.153
1231: fid: -0.994, c: 0.154
1232: fid: -0.994, c: 0.154
1233: fid: -0.994, c: 0.155
1234: fid: -0.994, c: 0.155
1235: fid: -0.994, c: 0.156
1236: fid: -0.994, c: 0.156
1237: fid: -0.994, c: 0.156
1238: fid: -0.994, c: 0.157
1239: fid: -0.994, c: 0.157
1240: fid: -0.994, c: 0.157
1241: fid: -0.994, c: 0.158
1242: fid: -0.994, c: 0.158
1243: fid: -0.994, c: 0.158
1244: fid: -0.994, c: 0.159
1245: fid: -0.994, c: 0.159
1246: fid: -0.994, c: 0.159
1247: fid: -0.994, c

1505: fid: -0.994, c: 0.140
1506: fid: -0.994, c: 0.140
1507: fid: -0.994, c: 0.140
1508: fid: -0.995, c: 0.140
1509: fid: -0.995, c: 0.140
1510: fid: -0.995, c: 0.140
1511: fid: -0.995, c: 0.140
1512: fid: -0.995, c: 0.140
1513: fid: -0.994, c: 0.140
1514: fid: -0.994, c: 0.140
1515: fid: -0.994, c: 0.140
1516: fid: -0.994, c: 0.139
1517: fid: -0.994, c: 0.139
1518: fid: -0.994, c: 0.138
1519: fid: -0.994, c: 0.138
1520: fid: -0.994, c: 0.138
1521: fid: -0.994, c: 0.138
1522: fid: -0.994, c: 0.138
1523: fid: -0.994, c: 0.138
1524: fid: -0.994, c: 0.138
1525: fid: -0.994, c: 0.138
1526: fid: -0.994, c: 0.138
1527: fid: -0.994, c: 0.138
1528: fid: -0.994, c: 0.138
1529: fid: -0.994, c: 0.138
1530: fid: -0.994, c: 0.138
1531: fid: -0.994, c: 0.138
1532: fid: -0.994, c: 0.138
1533: fid: -0.994, c: 0.138
1534: fid: -0.994, c: 0.138
1535: fid: -0.994, c: 0.138
1536: fid: -0.994, c: 0.138
1537: fid: -0.994, c: 0.138
1538: fid: -0.994, c: 0.138
1539: fid: -0.994, c: 0.138
1540: fid: -0.994, c

1799: fid: -0.995, c: 0.121
1800: fid: -0.995, c: 0.121
1801: fid: -0.995, c: 0.121
1802: fid: -0.995, c: 0.121
1803: fid: -0.995, c: 0.121
1804: fid: -0.995, c: 0.121
1805: fid: -0.995, c: 0.121
1806: fid: -0.995, c: 0.121
1807: fid: -0.995, c: 0.121
1808: fid: -0.995, c: 0.121
1809: fid: -0.995, c: 0.121
1810: fid: -0.995, c: 0.121
1811: fid: -0.995, c: 0.120
1812: fid: -0.995, c: 0.120
1813: fid: -0.995, c: 0.120
1814: fid: -0.995, c: 0.120
1815: fid: -0.995, c: 0.120
1816: fid: -0.995, c: 0.120
1817: fid: -0.995, c: 0.120
1818: fid: -0.995, c: 0.120
1819: fid: -0.995, c: 0.120
1820: fid: -0.995, c: 0.120
1821: fid: -0.995, c: 0.120
1822: fid: -0.995, c: 0.120
1823: fid: -0.995, c: 0.120
1824: fid: -0.995, c: 0.120
1825: fid: -0.995, c: 0.120
1826: fid: -0.995, c: 0.120
1827: fid: -0.995, c: 0.120
1828: fid: -0.995, c: 0.120
1829: fid: -0.995, c: 0.120
1830: fid: -0.995, c: 0.121
1831: fid: -0.995, c: 0.121
1832: fid: -0.995, c: 0.121
1833: fid: -0.995, c: 0.121
1834: fid: -0.995, c

#### Four Blocks

In [8]:
input_list, target_list = pickle.load(open("../../data/hsc_quad.p", "rb"))

n = 2
d = 2**n
rank = 8

circuit = qk.QuantumCircuit(2)
circuit.h(0)
circuit.s(1)
circuit.cnot(0,1)
circuit.h(0)
circuit.s(1)
circuit.cnot(0,1)
circuit.h(0)
circuit.s(1)
circuit.cnot(0,1)
circuit.h(0)
circuit.s(1)
circuit.cnot(0,1)

U = Operator(circuit).data

kraus_model = KrausMap(U, 0.5, d, rank)

model4 = ModelQuantumMap(q_map = kraus_model,
                         cost = state_density_loss,
                         input_list = input_list,
                         target_list = target_list,
                         lr = 0.05, 
                         h = 1e-3)

model4.train(num_iter = 2000, 
             use_adam = True)

  0%|          | 0/2000 [00:00<?, ?it/s]

0: fid: -0.822, c: 0.512
1: fid: -0.829, c: 0.525
2: fid: -0.836, c: 0.537
3: fid: -0.843, c: 0.550
4: fid: -0.849, c: 0.562
5: fid: -0.855, c: 0.574
6: fid: -0.861, c: 0.586
7: fid: -0.867, c: 0.597
8: fid: -0.872, c: 0.607
9: fid: -0.877, c: 0.618
10: fid: -0.881, c: 0.627
11: fid: -0.885, c: 0.637
12: fid: -0.890, c: 0.646
13: fid: -0.894, c: 0.656
14: fid: -0.898, c: 0.665
15: fid: -0.901, c: 0.674
16: fid: -0.905, c: 0.683
17: fid: -0.908, c: 0.691
18: fid: -0.911, c: 0.700
19: fid: -0.914, c: 0.707
20: fid: -0.916, c: 0.714
21: fid: -0.918, c: 0.721
22: fid: -0.920, c: 0.727
23: fid: -0.922, c: 0.733
24: fid: -0.924, c: 0.739
25: fid: -0.926, c: 0.744
26: fid: -0.927, c: 0.749
27: fid: -0.928, c: 0.754
28: fid: -0.930, c: 0.759
29: fid: -0.931, c: 0.763
30: fid: -0.932, c: 0.768
31: fid: -0.933, c: 0.772
32: fid: -0.934, c: 0.776
33: fid: -0.935, c: 0.780
34: fid: -0.936, c: 0.783
35: fid: -0.936, c: 0.787
36: fid: -0.937, c: 0.790
37: fid: -0.938, c: 0.793
38: fid: -0.938, c: 0.

308: fid: -0.973, c: 0.364
309: fid: -0.973, c: 0.362
310: fid: -0.973, c: 0.359
311: fid: -0.973, c: 0.359
312: fid: -0.974, c: 0.358
313: fid: -0.974, c: 0.358
314: fid: -0.974, c: 0.357
315: fid: -0.975, c: 0.357
316: fid: -0.975, c: 0.357
317: fid: -0.975, c: 0.356
318: fid: -0.975, c: 0.357
319: fid: -0.976, c: 0.357
320: fid: -0.976, c: 0.358
321: fid: -0.976, c: 0.358
322: fid: -0.976, c: 0.358
323: fid: -0.976, c: 0.358
324: fid: -0.976, c: 0.357
325: fid: -0.976, c: 0.356
326: fid: -0.976, c: 0.354
327: fid: -0.976, c: 0.352
328: fid: -0.976, c: 0.351
329: fid: -0.976, c: 0.351
330: fid: -0.976, c: 0.350
331: fid: -0.976, c: 0.350
332: fid: -0.976, c: 0.349
333: fid: -0.976, c: 0.348
334: fid: -0.976, c: 0.347
335: fid: -0.976, c: 0.345
336: fid: -0.976, c: 0.342
337: fid: -0.976, c: 0.340
338: fid: -0.976, c: 0.337
339: fid: -0.976, c: 0.335
340: fid: -0.976, c: 0.334
341: fid: -0.976, c: 0.332
342: fid: -0.976, c: 0.331
343: fid: -0.976, c: 0.329
344: fid: -0.976, c: 0.327
3

612: fid: -0.976, c: 0.194
613: fid: -0.976, c: 0.195
614: fid: -0.977, c: 0.195
615: fid: -0.977, c: 0.196
616: fid: -0.978, c: 0.197
617: fid: -0.978, c: 0.198
618: fid: -0.978, c: 0.199
619: fid: -0.978, c: 0.201
620: fid: -0.979, c: 0.202
621: fid: -0.979, c: 0.203
622: fid: -0.979, c: 0.205
623: fid: -0.979, c: 0.206
624: fid: -0.979, c: 0.206
625: fid: -0.979, c: 0.206
626: fid: -0.979, c: 0.206
627: fid: -0.979, c: 0.206
628: fid: -0.979, c: 0.206
629: fid: -0.979, c: 0.206
630: fid: -0.979, c: 0.206
631: fid: -0.979, c: 0.206
632: fid: -0.979, c: 0.206
633: fid: -0.979, c: 0.206
634: fid: -0.979, c: 0.206
635: fid: -0.979, c: 0.206
636: fid: -0.979, c: 0.206
637: fid: -0.979, c: 0.207
638: fid: -0.979, c: 0.208
639: fid: -0.979, c: 0.209
640: fid: -0.979, c: 0.210
641: fid: -0.979, c: 0.211
642: fid: -0.978, c: 0.212
643: fid: -0.978, c: 0.213
644: fid: -0.978, c: 0.214
645: fid: -0.978, c: 0.214
646: fid: -0.978, c: 0.214
647: fid: -0.978, c: 0.214
648: fid: -0.977, c: 0.214
6

916: fid: -0.980, c: 0.156
917: fid: -0.980, c: 0.155
918: fid: -0.980, c: 0.154
919: fid: -0.980, c: 0.153
920: fid: -0.980, c: 0.152
921: fid: -0.980, c: 0.151
922: fid: -0.980, c: 0.150
923: fid: -0.980, c: 0.150
924: fid: -0.980, c: 0.150
925: fid: -0.980, c: 0.149
926: fid: -0.980, c: 0.149
927: fid: -0.980, c: 0.149
928: fid: -0.980, c: 0.149
929: fid: -0.980, c: 0.149
930: fid: -0.980, c: 0.149
931: fid: -0.980, c: 0.148
932: fid: -0.980, c: 0.148
933: fid: -0.980, c: 0.148
934: fid: -0.980, c: 0.148
935: fid: -0.980, c: 0.148
936: fid: -0.979, c: 0.147
937: fid: -0.979, c: 0.147
938: fid: -0.979, c: 0.147
939: fid: -0.979, c: 0.146
940: fid: -0.979, c: 0.146
941: fid: -0.979, c: 0.146
942: fid: -0.979, c: 0.146
943: fid: -0.979, c: 0.146
944: fid: -0.979, c: 0.145
945: fid: -0.979, c: 0.145
946: fid: -0.979, c: 0.145
947: fid: -0.979, c: 0.145
948: fid: -0.979, c: 0.145
949: fid: -0.979, c: 0.145
950: fid: -0.980, c: 0.145
951: fid: -0.979, c: 0.144
952: fid: -0.979, c: 0.144
9

1212: fid: -0.980, c: 0.114
1213: fid: -0.980, c: 0.114
1214: fid: -0.980, c: 0.113
1215: fid: -0.980, c: 0.113
1216: fid: -0.980, c: 0.113
1217: fid: -0.980, c: 0.113
1218: fid: -0.980, c: 0.113
1219: fid: -0.980, c: 0.112
1220: fid: -0.980, c: 0.112
1221: fid: -0.980, c: 0.112
1222: fid: -0.980, c: 0.112
1223: fid: -0.980, c: 0.112
1224: fid: -0.980, c: 0.112
1225: fid: -0.980, c: 0.112
1226: fid: -0.980, c: 0.113
1227: fid: -0.980, c: 0.113
1228: fid: -0.980, c: 0.113
1229: fid: -0.980, c: 0.114
1230: fid: -0.980, c: 0.114
1231: fid: -0.980, c: 0.114
1232: fid: -0.980, c: 0.114
1233: fid: -0.981, c: 0.114
1234: fid: -0.981, c: 0.114
1235: fid: -0.981, c: 0.114
1236: fid: -0.981, c: 0.113
1237: fid: -0.981, c: 0.113
1238: fid: -0.981, c: 0.113
1239: fid: -0.981, c: 0.113
1240: fid: -0.981, c: 0.113
1241: fid: -0.981, c: 0.113
1242: fid: -0.981, c: 0.113
1243: fid: -0.981, c: 0.114
1244: fid: -0.981, c: 0.114
1245: fid: -0.981, c: 0.114
1246: fid: -0.981, c: 0.114
1247: fid: -0.981, c

1505: fid: -0.981, c: 0.096
1506: fid: -0.981, c: 0.095
1507: fid: -0.981, c: 0.095
1508: fid: -0.981, c: 0.095
1509: fid: -0.981, c: 0.095
1510: fid: -0.981, c: 0.094
1511: fid: -0.981, c: 0.094
1512: fid: -0.981, c: 0.094
1513: fid: -0.981, c: 0.094
1514: fid: -0.981, c: 0.094
1515: fid: -0.981, c: 0.093
1516: fid: -0.981, c: 0.093
1517: fid: -0.981, c: 0.093
1518: fid: -0.981, c: 0.093
1519: fid: -0.981, c: 0.093
1520: fid: -0.981, c: 0.092
1521: fid: -0.981, c: 0.092
1522: fid: -0.981, c: 0.092
1523: fid: -0.981, c: 0.092
1524: fid: -0.981, c: 0.092
1525: fid: -0.981, c: 0.092
1526: fid: -0.981, c: 0.092
1527: fid: -0.981, c: 0.092
1528: fid: -0.981, c: 0.093
1529: fid: -0.981, c: 0.093
1530: fid: -0.981, c: 0.094
1531: fid: -0.981, c: 0.094
1532: fid: -0.981, c: 0.094
1533: fid: -0.981, c: 0.094
1534: fid: -0.981, c: 0.095
1535: fid: -0.981, c: 0.095
1536: fid: -0.981, c: 0.095
1537: fid: -0.981, c: 0.095
1538: fid: -0.981, c: 0.095
1539: fid: -0.981, c: 0.095
1540: fid: -0.981, c

1798: fid: -0.980, c: 0.082
1799: fid: -0.981, c: 0.082
1800: fid: -0.981, c: 0.082
1801: fid: -0.981, c: 0.082
1802: fid: -0.981, c: 0.082
1803: fid: -0.981, c: 0.082
1804: fid: -0.981, c: 0.082
1805: fid: -0.981, c: 0.082
1806: fid: -0.980, c: 0.083
1807: fid: -0.980, c: 0.083
1808: fid: -0.980, c: 0.083
1809: fid: -0.980, c: 0.083
1810: fid: -0.980, c: 0.083
1811: fid: -0.980, c: 0.082
1812: fid: -0.980, c: 0.082
1813: fid: -0.980, c: 0.082
1814: fid: -0.980, c: 0.082
1815: fid: -0.980, c: 0.081
1816: fid: -0.980, c: 0.081
1817: fid: -0.980, c: 0.081
1818: fid: -0.980, c: 0.081
1819: fid: -0.980, c: 0.081
1820: fid: -0.980, c: 0.081
1821: fid: -0.980, c: 0.081
1822: fid: -0.980, c: 0.081
1823: fid: -0.980, c: 0.081
1824: fid: -0.980, c: 0.081
1825: fid: -0.980, c: 0.081
1826: fid: -0.980, c: 0.082
1827: fid: -0.980, c: 0.082
1828: fid: -0.980, c: 0.082
1829: fid: -0.980, c: 0.082
1830: fid: -0.980, c: 0.082
1831: fid: -0.980, c: 0.082
1832: fid: -0.980, c: 0.082
1833: fid: -0.980, c

#### Five

In [9]:
input_list, target_list = pickle.load(open("../../data/hsc_five.p", "rb"))

n = 2
d = 2**n
rank = 9

circuit = qk.QuantumCircuit(2)
circuit.h(0)
circuit.s(1)
circuit.cnot(0,1)
circuit.h(0)
circuit.s(1)
circuit.cnot(0,1)
circuit.h(0)
circuit.s(1)
circuit.cnot(0,1)
circuit.h(0)
circuit.s(1)
circuit.cnot(0,1)
circuit.h(0)
circuit.s(1)
circuit.cnot(0,1)

U = Operator(circuit).data

kraus_model = KrausMap(U, 0.5, d, rank)

model5 = ModelQuantumMap(q_map = kraus_model,
                         cost = state_density_loss,
                         input_list = input_list,
                         target_list = target_list,
                         lr = 0.05, 
                         h = 1e-3)

model5.train(num_iter = 2000, 
             use_adam = True)

  0%|          | 0/2000 [00:00<?, ?it/s]

0: fid: -0.840, c: 0.512
1: fid: -0.848, c: 0.525
2: fid: -0.855, c: 0.537
3: fid: -0.862, c: 0.550
4: fid: -0.868, c: 0.562
5: fid: -0.874, c: 0.573
6: fid: -0.880, c: 0.584
7: fid: -0.885, c: 0.596
8: fid: -0.890, c: 0.606
9: fid: -0.895, c: 0.617
10: fid: -0.900, c: 0.628
11: fid: -0.905, c: 0.639
12: fid: -0.909, c: 0.649
13: fid: -0.913, c: 0.659
14: fid: -0.917, c: 0.669
15: fid: -0.920, c: 0.679
16: fid: -0.923, c: 0.688
17: fid: -0.926, c: 0.697
18: fid: -0.929, c: 0.706
19: fid: -0.931, c: 0.714
20: fid: -0.933, c: 0.722
21: fid: -0.935, c: 0.729
22: fid: -0.937, c: 0.737
23: fid: -0.939, c: 0.744
24: fid: -0.941, c: 0.751
25: fid: -0.942, c: 0.758
26: fid: -0.943, c: 0.764
27: fid: -0.944, c: 0.769
28: fid: -0.945, c: 0.774
29: fid: -0.946, c: 0.779
30: fid: -0.947, c: 0.784
31: fid: -0.948, c: 0.788
32: fid: -0.948, c: 0.792
33: fid: -0.949, c: 0.795
34: fid: -0.949, c: 0.798
35: fid: -0.950, c: 0.801
36: fid: -0.950, c: 0.804
37: fid: -0.951, c: 0.806
38: fid: -0.951, c: 0.

308: fid: -0.993, c: 0.302
309: fid: -0.993, c: 0.300
310: fid: -0.993, c: 0.299
311: fid: -0.993, c: 0.298
312: fid: -0.993, c: 0.297
313: fid: -0.993, c: 0.296
314: fid: -0.993, c: 0.294
315: fid: -0.993, c: 0.294
316: fid: -0.993, c: 0.292
317: fid: -0.993, c: 0.291
318: fid: -0.993, c: 0.291
319: fid: -0.993, c: 0.290
320: fid: -0.993, c: 0.290
321: fid: -0.993, c: 0.289
322: fid: -0.993, c: 0.288
323: fid: -0.993, c: 0.288
324: fid: -0.993, c: 0.287
325: fid: -0.993, c: 0.287
326: fid: -0.993, c: 0.287
327: fid: -0.993, c: 0.287
328: fid: -0.993, c: 0.287
329: fid: -0.993, c: 0.286
330: fid: -0.993, c: 0.286
331: fid: -0.993, c: 0.285
332: fid: -0.993, c: 0.285
333: fid: -0.993, c: 0.284
334: fid: -0.993, c: 0.284
335: fid: -0.993, c: 0.283
336: fid: -0.993, c: 0.282
337: fid: -0.993, c: 0.281
338: fid: -0.993, c: 0.280
339: fid: -0.993, c: 0.279
340: fid: -0.993, c: 0.279
341: fid: -0.993, c: 0.278
342: fid: -0.993, c: 0.277
343: fid: -0.993, c: 0.276
344: fid: -0.993, c: 0.275
3

612: fid: -0.994, c: 0.183
613: fid: -0.994, c: 0.183
614: fid: -0.994, c: 0.182
615: fid: -0.994, c: 0.181
616: fid: -0.994, c: 0.181
617: fid: -0.994, c: 0.180
618: fid: -0.994, c: 0.179
619: fid: -0.994, c: 0.178
620: fid: -0.994, c: 0.178
621: fid: -0.994, c: 0.177
622: fid: -0.994, c: 0.176
623: fid: -0.994, c: 0.175
624: fid: -0.994, c: 0.175
625: fid: -0.994, c: 0.175
626: fid: -0.994, c: 0.175
627: fid: -0.994, c: 0.175
628: fid: -0.994, c: 0.174
629: fid: -0.994, c: 0.174
630: fid: -0.994, c: 0.173
631: fid: -0.994, c: 0.173
632: fid: -0.994, c: 0.172
633: fid: -0.994, c: 0.172
634: fid: -0.994, c: 0.172
635: fid: -0.994, c: 0.172
636: fid: -0.994, c: 0.171
637: fid: -0.994, c: 0.171
638: fid: -0.994, c: 0.170
639: fid: -0.994, c: 0.170
640: fid: -0.994, c: 0.170
641: fid: -0.994, c: 0.169
642: fid: -0.994, c: 0.169
643: fid: -0.994, c: 0.168
644: fid: -0.994, c: 0.168
645: fid: -0.995, c: 0.168
646: fid: -0.995, c: 0.167
647: fid: -0.994, c: 0.167
648: fid: -0.994, c: 0.167
6

916: fid: -0.995, c: 0.148
917: fid: -0.995, c: 0.148
918: fid: -0.995, c: 0.149
919: fid: -0.995, c: 0.149
920: fid: -0.995, c: 0.149
921: fid: -0.995, c: 0.149
922: fid: -0.995, c: 0.149
923: fid: -0.995, c: 0.149
924: fid: -0.995, c: 0.149
925: fid: -0.995, c: 0.149
926: fid: -0.995, c: 0.149
927: fid: -0.995, c: 0.149
928: fid: -0.995, c: 0.149
929: fid: -0.995, c: 0.149
930: fid: -0.995, c: 0.149
931: fid: -0.995, c: 0.149
932: fid: -0.995, c: 0.148
933: fid: -0.995, c: 0.148
934: fid: -0.995, c: 0.148
935: fid: -0.995, c: 0.147
936: fid: -0.995, c: 0.147
937: fid: -0.995, c: 0.146
938: fid: -0.995, c: 0.146
939: fid: -0.995, c: 0.145
940: fid: -0.994, c: 0.145
941: fid: -0.994, c: 0.145
942: fid: -0.994, c: 0.145
943: fid: -0.995, c: 0.145
944: fid: -0.995, c: 0.145
945: fid: -0.995, c: 0.145
946: fid: -0.995, c: 0.145
947: fid: -0.995, c: 0.146
948: fid: -0.995, c: 0.146
949: fid: -0.995, c: 0.146
950: fid: -0.995, c: 0.146
951: fid: -0.995, c: 0.146
952: fid: -0.995, c: 0.146
9

1212: fid: -0.995, c: 0.131
1213: fid: -0.995, c: 0.131
1214: fid: -0.995, c: 0.131
1215: fid: -0.995, c: 0.130
1216: fid: -0.995, c: 0.130
1217: fid: -0.995, c: 0.131
1218: fid: -0.995, c: 0.131
1219: fid: -0.995, c: 0.131
1220: fid: -0.995, c: 0.131
1221: fid: -0.995, c: 0.132
1222: fid: -0.995, c: 0.132
1223: fid: -0.995, c: 0.133
1224: fid: -0.995, c: 0.133
1225: fid: -0.995, c: 0.133
1226: fid: -0.995, c: 0.134
1227: fid: -0.995, c: 0.134
1228: fid: -0.995, c: 0.134
1229: fid: -0.995, c: 0.134
1230: fid: -0.995, c: 0.135
1231: fid: -0.995, c: 0.135
1232: fid: -0.995, c: 0.135
1233: fid: -0.994, c: 0.135
1234: fid: -0.994, c: 0.135
1235: fid: -0.994, c: 0.135
1236: fid: -0.994, c: 0.135
1237: fid: -0.994, c: 0.135
1238: fid: -0.994, c: 0.134
1239: fid: -0.994, c: 0.134
1240: fid: -0.994, c: 0.134
1241: fid: -0.994, c: 0.133
1242: fid: -0.994, c: 0.133
1243: fid: -0.994, c: 0.133
1244: fid: -0.994, c: 0.133
1245: fid: -0.994, c: 0.133
1246: fid: -0.994, c: 0.133
1247: fid: -0.994, c

1505: fid: -0.994, c: 0.124
1506: fid: -0.994, c: 0.124
1507: fid: -0.994, c: 0.124
1508: fid: -0.994, c: 0.124
1509: fid: -0.994, c: 0.123
1510: fid: -0.994, c: 0.123
1511: fid: -0.994, c: 0.122
1512: fid: -0.994, c: 0.122
1513: fid: -0.994, c: 0.121
1514: fid: -0.994, c: 0.121
1515: fid: -0.994, c: 0.121
1516: fid: -0.995, c: 0.121
1517: fid: -0.995, c: 0.120
1518: fid: -0.995, c: 0.121
1519: fid: -0.995, c: 0.120
1520: fid: -0.995, c: 0.120
1521: fid: -0.995, c: 0.120
1522: fid: -0.995, c: 0.120
1523: fid: -0.995, c: 0.120
1524: fid: -0.995, c: 0.120
1525: fid: -0.995, c: 0.121
1526: fid: -0.995, c: 0.121
1527: fid: -0.995, c: 0.122
1528: fid: -0.995, c: 0.122
1529: fid: -0.995, c: 0.123
1530: fid: -0.995, c: 0.124
1531: fid: -0.995, c: 0.124
1532: fid: -0.995, c: 0.125
1533: fid: -0.995, c: 0.125
1534: fid: -0.995, c: 0.126
1535: fid: -0.995, c: 0.126
1536: fid: -0.995, c: 0.127
1537: fid: -0.995, c: 0.127
1538: fid: -0.995, c: 0.128
1539: fid: -0.995, c: 0.128
1540: fid: -0.995, c

1798: fid: -0.995, c: 0.115
1799: fid: -0.995, c: 0.115
1800: fid: -0.995, c: 0.115
1801: fid: -0.995, c: 0.115
1802: fid: -0.995, c: 0.114
1803: fid: -0.995, c: 0.114
1804: fid: -0.995, c: 0.114
1805: fid: -0.995, c: 0.115
1806: fid: -0.995, c: 0.115
1807: fid: -0.995, c: 0.115
1808: fid: -0.995, c: 0.115
1809: fid: -0.995, c: 0.115
1810: fid: -0.995, c: 0.115
1811: fid: -0.995, c: 0.115
1812: fid: -0.995, c: 0.115
1813: fid: -0.995, c: 0.115
1814: fid: -0.995, c: 0.115
1815: fid: -0.995, c: 0.115
1816: fid: -0.994, c: 0.115
1817: fid: -0.994, c: 0.114
1818: fid: -0.994, c: 0.114
1819: fid: -0.994, c: 0.114
1820: fid: -0.995, c: 0.114
1821: fid: -0.995, c: 0.114
1822: fid: -0.995, c: 0.113
1823: fid: -0.995, c: 0.113
1824: fid: -0.995, c: 0.113
1825: fid: -0.995, c: 0.112
1826: fid: -0.995, c: 0.112
1827: fid: -0.995, c: 0.112
1828: fid: -0.995, c: 0.111
1829: fid: -0.995, c: 0.111
1830: fid: -0.995, c: 0.111
1831: fid: -0.995, c: 0.111
1832: fid: -0.995, c: 0.111
1833: fid: -0.995, c

#### Six

In [12]:
input_list, target_list = pickle.load(open("../../data/hsc_six.p", "rb"))

n = 2
d = 2**n
rank = 10

circuit = qk.QuantumCircuit(2)
circuit.h(0)
circuit.s(1)
circuit.cnot(0,1)
circuit.h(0)
circuit.s(1)
circuit.cnot(0,1)
circuit.h(0)
circuit.s(1)
circuit.cnot(0,1)
circuit.h(0)
circuit.s(1)
circuit.cnot(0,1)
circuit.h(0)
circuit.s(1)
circuit.cnot(0,1)
circuit.h(0)
circuit.s(1)
circuit.cnot(0,1)

U = Operator(circuit).data

kraus_model = KrausMap(U, 0.5, d, rank)

model6 = ModelQuantumMap(q_map = kraus_model,
                         cost = state_density_loss,
                         input_list = input_list,
                         target_list = target_list,
                         lr = 0.05, 
                         h = 1e-3)

model6.train(num_iter = 2000, 
             use_adam = True)

  0%|          | 0/2000 [00:00<?, ?it/s]

0: fid: -0.800, c: 0.512
1: fid: -0.808, c: 0.525
2: fid: -0.815, c: 0.537
3: fid: -0.823, c: 0.549
4: fid: -0.830, c: 0.561
5: fid: -0.837, c: 0.572
6: fid: -0.844, c: 0.584
7: fid: -0.851, c: 0.595
8: fid: -0.857, c: 0.606
9: fid: -0.863, c: 0.618
10: fid: -0.869, c: 0.629
11: fid: -0.874, c: 0.640
12: fid: -0.879, c: 0.650
13: fid: -0.884, c: 0.660
14: fid: -0.889, c: 0.670
15: fid: -0.893, c: 0.680
16: fid: -0.897, c: 0.689
17: fid: -0.900, c: 0.698
18: fid: -0.903, c: 0.707
19: fid: -0.907, c: 0.715
20: fid: -0.909, c: 0.723
21: fid: -0.912, c: 0.730
22: fid: -0.915, c: 0.738
23: fid: -0.917, c: 0.745
24: fid: -0.919, c: 0.752
25: fid: -0.921, c: 0.758
26: fid: -0.923, c: 0.765
27: fid: -0.925, c: 0.771
28: fid: -0.927, c: 0.776
29: fid: -0.928, c: 0.782
30: fid: -0.930, c: 0.788
31: fid: -0.931, c: 0.793
32: fid: -0.932, c: 0.797
33: fid: -0.933, c: 0.802
34: fid: -0.934, c: 0.806
35: fid: -0.934, c: 0.809
36: fid: -0.935, c: 0.813
37: fid: -0.936, c: 0.816
38: fid: -0.936, c: 0.

308: fid: -0.989, c: 0.268
309: fid: -0.990, c: 0.267
310: fid: -0.990, c: 0.266
311: fid: -0.990, c: 0.266
312: fid: -0.990, c: 0.265
313: fid: -0.990, c: 0.264
314: fid: -0.990, c: 0.263
315: fid: -0.990, c: 0.263
316: fid: -0.990, c: 0.262
317: fid: -0.990, c: 0.261
318: fid: -0.990, c: 0.260
319: fid: -0.990, c: 0.260
320: fid: -0.990, c: 0.259
321: fid: -0.990, c: 0.259
322: fid: -0.990, c: 0.258
323: fid: -0.990, c: 0.257
324: fid: -0.990, c: 0.256
325: fid: -0.990, c: 0.255
326: fid: -0.990, c: 0.254
327: fid: -0.990, c: 0.253
328: fid: -0.990, c: 0.252
329: fid: -0.990, c: 0.251
330: fid: -0.990, c: 0.250
331: fid: -0.990, c: 0.249
332: fid: -0.990, c: 0.248
333: fid: -0.990, c: 0.247
334: fid: -0.990, c: 0.247
335: fid: -0.990, c: 0.246
336: fid: -0.990, c: 0.245
337: fid: -0.990, c: 0.244
338: fid: -0.990, c: 0.243
339: fid: -0.991, c: 0.242
340: fid: -0.991, c: 0.241
341: fid: -0.991, c: 0.240
342: fid: -0.991, c: 0.238
343: fid: -0.991, c: 0.237
344: fid: -0.991, c: 0.236
3

612: fid: -0.993, c: 0.130
613: fid: -0.993, c: 0.130
614: fid: -0.993, c: 0.130
615: fid: -0.993, c: 0.130
616: fid: -0.993, c: 0.130
617: fid: -0.993, c: 0.130
618: fid: -0.993, c: 0.130
619: fid: -0.993, c: 0.130
620: fid: -0.993, c: 0.130
621: fid: -0.993, c: 0.130
622: fid: -0.993, c: 0.130
623: fid: -0.993, c: 0.129
624: fid: -0.993, c: 0.129
625: fid: -0.993, c: 0.129
626: fid: -0.994, c: 0.128
627: fid: -0.994, c: 0.128
628: fid: -0.993, c: 0.128
629: fid: -0.993, c: 0.127
630: fid: -0.993, c: 0.127
631: fid: -0.993, c: 0.127
632: fid: -0.993, c: 0.126
633: fid: -0.993, c: 0.126
634: fid: -0.993, c: 0.126
635: fid: -0.993, c: 0.126
636: fid: -0.993, c: 0.126
637: fid: -0.993, c: 0.126
638: fid: -0.993, c: 0.126
639: fid: -0.993, c: 0.126
640: fid: -0.993, c: 0.126
641: fid: -0.993, c: 0.126
642: fid: -0.993, c: 0.126
643: fid: -0.993, c: 0.126
644: fid: -0.992, c: 0.126
645: fid: -0.992, c: 0.125
646: fid: -0.993, c: 0.126
647: fid: -0.993, c: 0.126
648: fid: -0.993, c: 0.126
6

916: fid: -0.994, c: 0.099
917: fid: -0.994, c: 0.099
918: fid: -0.994, c: 0.099
919: fid: -0.994, c: 0.099
920: fid: -0.994, c: 0.099
921: fid: -0.994, c: 0.099
922: fid: -0.994, c: 0.099
923: fid: -0.994, c: 0.099
924: fid: -0.994, c: 0.098
925: fid: -0.994, c: 0.098
926: fid: -0.994, c: 0.098
927: fid: -0.994, c: 0.098
928: fid: -0.994, c: 0.098
929: fid: -0.994, c: 0.097
930: fid: -0.994, c: 0.097
931: fid: -0.994, c: 0.097
932: fid: -0.994, c: 0.097
933: fid: -0.994, c: 0.097
934: fid: -0.994, c: 0.096
935: fid: -0.993, c: 0.096
936: fid: -0.993, c: 0.096
937: fid: -0.993, c: 0.096
938: fid: -0.994, c: 0.096
939: fid: -0.994, c: 0.096
940: fid: -0.994, c: 0.096
941: fid: -0.994, c: 0.096
942: fid: -0.994, c: 0.097
943: fid: -0.994, c: 0.097
944: fid: -0.994, c: 0.097
945: fid: -0.994, c: 0.097
946: fid: -0.994, c: 0.097
947: fid: -0.994, c: 0.097
948: fid: -0.994, c: 0.097
949: fid: -0.994, c: 0.097
950: fid: -0.993, c: 0.097
951: fid: -0.993, c: 0.097
952: fid: -0.993, c: 0.097
9

1212: fid: -0.993, c: 0.079
1213: fid: -0.993, c: 0.079
1214: fid: -0.993, c: 0.079
1215: fid: -0.993, c: 0.079
1216: fid: -0.993, c: 0.079
1217: fid: -0.993, c: 0.079
1218: fid: -0.993, c: 0.079
1219: fid: -0.993, c: 0.079
1220: fid: -0.993, c: 0.079
1221: fid: -0.994, c: 0.079
1222: fid: -0.994, c: 0.079
1223: fid: -0.994, c: 0.079
1224: fid: -0.994, c: 0.079
1225: fid: -0.994, c: 0.079
1226: fid: -0.994, c: 0.079
1227: fid: -0.993, c: 0.079
1228: fid: -0.993, c: 0.079
1229: fid: -0.993, c: 0.079
1230: fid: -0.993, c: 0.079
1231: fid: -0.993, c: 0.080
1232: fid: -0.993, c: 0.080
1233: fid: -0.993, c: 0.080
1234: fid: -0.993, c: 0.080
1235: fid: -0.994, c: 0.080
1236: fid: -0.994, c: 0.080
1237: fid: -0.994, c: 0.079
1238: fid: -0.994, c: 0.079
1239: fid: -0.994, c: 0.079
1240: fid: -0.994, c: 0.079
1241: fid: -0.994, c: 0.079
1242: fid: -0.994, c: 0.079
1243: fid: -0.994, c: 0.079
1244: fid: -0.993, c: 0.079
1245: fid: -0.993, c: 0.079
1246: fid: -0.993, c: 0.079
1247: fid: -0.993, c

1505: fid: -0.994, c: 0.066
1506: fid: -0.994, c: 0.066
1507: fid: -0.994, c: 0.066
1508: fid: -0.994, c: 0.066
1509: fid: -0.994, c: 0.066
1510: fid: -0.994, c: 0.066
1511: fid: -0.994, c: 0.066
1512: fid: -0.994, c: 0.066
1513: fid: -0.994, c: 0.066
1514: fid: -0.994, c: 0.066
1515: fid: -0.994, c: 0.066
1516: fid: -0.994, c: 0.066
1517: fid: -0.994, c: 0.066
1518: fid: -0.994, c: 0.067
1519: fid: -0.994, c: 0.067
1520: fid: -0.994, c: 0.067
1521: fid: -0.994, c: 0.067
1522: fid: -0.994, c: 0.067
1523: fid: -0.994, c: 0.067
1524: fid: -0.994, c: 0.067
1525: fid: -0.994, c: 0.067
1526: fid: -0.994, c: 0.067
1527: fid: -0.994, c: 0.067
1528: fid: -0.994, c: 0.067
1529: fid: -0.994, c: 0.067
1530: fid: -0.994, c: 0.067
1531: fid: -0.994, c: 0.067
1532: fid: -0.994, c: 0.067
1533: fid: -0.994, c: 0.067
1534: fid: -0.994, c: 0.067
1535: fid: -0.994, c: 0.067
1536: fid: -0.994, c: 0.067
1537: fid: -0.994, c: 0.067
1538: fid: -0.994, c: 0.067
1539: fid: -0.994, c: 0.067
1540: fid: -0.994, c

1798: fid: -0.994, c: 0.060
1799: fid: -0.994, c: 0.060
1800: fid: -0.994, c: 0.060
1801: fid: -0.994, c: 0.060
1802: fid: -0.994, c: 0.060
1803: fid: -0.994, c: 0.060
1804: fid: -0.994, c: 0.060
1805: fid: -0.994, c: 0.060
1806: fid: -0.994, c: 0.060
1807: fid: -0.994, c: 0.060
1808: fid: -0.994, c: 0.060
1809: fid: -0.994, c: 0.060
1810: fid: -0.994, c: 0.060
1811: fid: -0.994, c: 0.060
1812: fid: -0.994, c: 0.060
1813: fid: -0.994, c: 0.060
1814: fid: -0.994, c: 0.060
1815: fid: -0.994, c: 0.060
1816: fid: -0.994, c: 0.060
1817: fid: -0.994, c: 0.060
1818: fid: -0.994, c: 0.060
1819: fid: -0.994, c: 0.060
1820: fid: -0.994, c: 0.060
1821: fid: -0.994, c: 0.060
1822: fid: -0.994, c: 0.060
1823: fid: -0.994, c: 0.060
1824: fid: -0.994, c: 0.060
1825: fid: -0.994, c: 0.060
1826: fid: -0.993, c: 0.060
1827: fid: -0.993, c: 0.060
1828: fid: -0.993, c: 0.060
1829: fid: -0.993, c: 0.060
1830: fid: -0.993, c: 0.061
1831: fid: -0.993, c: 0.061
1832: fid: -0.993, c: 0.061
1833: fid: -0.993, c

In [ ]:
input_list, target_list = pickle.load(open("..\..\data\hsc_seven.p", "rb"))

n = 2
d = 2**n
rank = 10

circuit = qk.QuantumCircuit(2)
circuit.h(0)
circuit.s(1)
circuit.cnot(0,1)
circuit.h(0)
circuit.s(1)
circuit.cnot(0,1)
circuit.h(0)
circuit.s(1)
circuit.cnot(0,1)
circuit.h(0)
circuit.s(1)
circuit.cnot(0,1)
circuit.h(0)
circuit.s(1)
circuit.cnot(0,1)
circuit.h(0)
circuit.s(1)
circuit.cnot(0,1)
circuit.h(0)
circuit.s(1)
circuit.cnot(0,1)

U = Operator(circuit).data

kraus_model = KrausMap(U, 0.5, d, rank)

model7 = ModelQuantumMap(q_map = kraus_model,
                         cost = state_density_loss,
                         input_list = input_list,
                         target_list = target_list,
                         lr = 0.05, 
                         h = 1e-3)

model7.train(num_iter = 2000, 
             use_adam = True)

In [13]:
n_samples = 1000
fid_array = np.zeros((n_samples, 6))

np.random.seed(42)
for i in range(n_samples):
    state = generate_state(d,1)
    
    state_model1   = model1.q_map.apply_map(state)
    state_model1_2 = model1.q_map.apply_map(state_model1)
    state_model1_3 = model1.q_map.apply_map(state_model1_2)
    state_model1_4 = model1.q_map.apply_map(state_model1_3)
    state_model1_5 = model1.q_map.apply_map(state_model1_4)
    state_model1_6 = model1.q_map.apply_map(state_model1_5)
    #state_model1_7 = model1.q_map.apply_map(state_model1_6)
    
    state_model2 = model2.q_map.apply_map(state)
    state_model3 = model3.q_map.apply_map(state)
    state_model4 = model4.q_map.apply_map(state)
    state_model5 = model5.q_map.apply_map(state)
    state_model6 = model6.q_map.apply_map(state)
    #state_model7 = model4.q_map.apply_map(state)
    
    fid_array[i, 0] = state_fidelity(state_model1_2, state_model2)
    fid_array[i, 1] = state_fidelity(state_model1_3, state_model3)
    fid_array[i, 2] = state_fidelity(state_model1_4, state_model4)
    fid_array[i, 3] = state_fidelity(state_model1_5, state_model5)
    fid_array[i, 4] = state_fidelity(state_model1_6, state_model6)
    #fid_array[i, 5] = state_fidelity(state_model1_7, state_model7)
    
    
fid_mean = np.mean(fid_array, axis=0)
fid_std = np.std(fid_array, axis=0)

print(fid_mean)
print(fid_std)

[0.97347756 0.9574411  0.94982185 0.93789337 0.91415761 0.        ]
[0.00879162 0.01082568 0.0131931  0.01623982 0.01677008 0.        ]
